In [32]:
import numpy as np
import pandas as pd


lex = pd.read_csv('Lexique380.utf8.csv')
#lex = pd.read_csv('test.csv')
lex = lex[(lex.ortho.str.contains('-| ') == False) & (lex.phon.str.contains('°') == False)]  # suppress schwa
lex = lex.drop_duplicates(subset='ortho', keep="first")
lex = lex[['ortho', 'p_cvcv','nbhomogr']]
dic = lex.set_index('ortho').to_dict()

p_dic = dic['p_cvcv']
n_dic = dic['nbhomogr']


In [33]:
def get_LPC_g(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace('.', '')
    snt_to_wlst = sentence.split(" ")
    LPC_gustures = 0 
    e = 0
    no_in_db = []
    more_ortho ={}

    
    for word in snt_to_wlst:
        if word in p_dic:
            #print(word, p_dic[word])
            e = 1
            if p_dic[word][0] == 'V':
                LPC_gustures += 1
            for letter in p_dic[word]:
                if letter == 'C':
                    LPC_gustures += 1
            if n_dic[word] != 1:
                more_ortho[word] = p_dic[word]

        else: 
            e = 0
        if e == 0 :
            no_in_db.append(word)
        
    
    if no_in_db == [] and more_ortho == []:
        return f'{LPC_gustures}'
    if no_in_db == [] and more_ortho != []:
        return f'{LPC_gustures}, check the cv for: {more_ortho}'
    else:
        return f'{LPC_gustures}, except {no_in_db}, check the cv for: {more_ortho}'


    


In [34]:
def get_number_of_letters(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace(' ', '')
    return len(sentence)
    

## Counting the numbers of LPC gestures for both sentences in to the file

In [35]:
df = pd.read_csv('CUSTIME_sentence_predictability.csv', encoding="utf-8")

for index,row in df.iterrows():
    p_sentence = row['Predictable end sentence']
    pred_end_n_LPC = get_LPC_g(p_sentence)
    pred_num_ltr = get_number_of_letters(p_sentence)
    df.loc[index,"pred_num_letters"] = pred_num_ltr
    df.loc[index,"pred_end_n_LPC"] = pred_end_n_LPC
    
    unp_sentence = row['Unpredictable end sentence']
    unpred_end_n_LPC = get_LPC_g(unp_sentence)
    unpred_num_ltr = get_number_of_letters(unp_sentence)
    df.loc[index,"unpred_num_letters"] = unpred_num_ltr
    df.loc[index,"unpred_end_n_LPC"] = unpred_end_n_LPC
    

df.to_csv('CUSTIME_sentence_predictability_with_LPC.csv', encoding="utf-8-sig")



